In [3]:
import pandas as pd

# Load the datasets
customers_df = pd.read_csv('/content/customers.csv')
marketing_interactions_df = pd.read_csv('/content/marketing_interactions.csv')
subscriptions_df = pd.read_csv('/content/subscriptions.csv')
transactions_df = pd.read_csv('/content/transactions.csv')

# Display the first few rows of each dataframe to understand their structure
customers_df.head(), marketing_interactions_df.head(), subscriptions_df.head(), transactions_df.head()


(   CustomerID SignUpDate  Age Gender  Income     Location
 0           1    8/12/21   67      F   43693      Chicago
 1           2    4/10/20   29      M   54219  Los Angeles
 2           3    11/5/20   30      F   31807      Chicago
 3           4    4/14/21   69      M   90561      Chicago
 4           5    11/7/21   49      F   30544       Boston,
    InterationID  CustomerID  CampaignID InteractionDate  Response
 0             1         146           6      2021-01-16         1
 1             2         987          33      2020-11-07         1
 2             3         315          29      2020-11-15         0
 3             4         345          19      2021-12-30         1
 4             5          54          15      2022-11-20         1,
    CustomerID  SubscriptionID   StartDate     EndDate  Churned
 0           1               1  2021-08-12         NaN        1
 1           2               2  2020-04-10         NaN        0
 2           3               3  2020-11-05  2021-1

In [4]:
# Merge the datasets on CustomerID to link marketing interactions with transactions and subscriptions
merged_df = marketing_interactions_df.merge(transactions_df, on='CustomerID', how='left') \
                                     .merge(subscriptions_df, on='CustomerID', how='left') \
                                     .merge(customers_df, on='CustomerID', how='left')

# Display the first few rows of the merged dataframe to verify the merge
merged_df.head()


,InterationID,CustomerID,CampaignID,InteractionDate,Response,TransactionID,PurchaseDate,Amount,ProductID,SubscriptionID,StartDate,EndDate,Churned,SignUpDate,Age,Gender,Income,Location
0,1,146,6,2021-01-16,1,481.0,2020-12-08,358.930080,38.0,146,2021-03-08,NaN,0,3/8/21,69,F,99329,Los Angeles
1,1,146,6,2021-01-16,1,1577.0,2022-02-08,13.044249,8.0,146,2021-03-08,NaN,0,3/8/21,69,F,99329,Los Angeles
2,1,146,6,2021-01-16,1,2332.0,2022-12-28,63.157793,47.0,146,2021-03-08,NaN,0,3/8/21,69,F,99329,Los Angeles
3,2,987,33,2020-11-07,1,8.0,2022-10-06,480.124605,44.0,987,2021-07-15,NaN,0,7/15/21,19,M,44415,Boston
4,2,987,33,2020-11-07,1,3393.0,2020-05-27,310.222629,32.0,987,2021-07-15,NaN,0,7/15/21,19,M,44415,Boston


In [5]:
# Check the correlation between 'Response' to marketing interactions and 'Amount' of transactions
response_vs_amount_correlation = merged_df[['Response', 'Amount']].corr()

# Group by Response and calculate the average transaction amount
avg_transaction_by_response = merged_df.groupby('Response')['Amount'].mean()

# Group by Response and calculate the churn rate
churn_rate_by_response = merged_df.groupby('Response')['Churned'].mean()

# Display the correlation, average transaction by response, and churn rate by response
response_vs_amount_correlation, avg_transaction_by_response, churn_rate_by_response


(          Response    Amount
 Response  1.000000  0.005276
 Amount    0.005276  1.000000,
 Response
 0    254.121660
 1    255.625819
 Name: Amount, dtype: float64,
 Response
 0    0.299300
 1    0.283529
 Name: Churned, dtype: float64)

In [6]:
# Analyze which campaigns were the most successful by calculating response rates for each campaign

# Group by CampaignID and calculate the response rate (i.e., percentage of positive responses)
campaign_success = merged_df.groupby('CampaignID')['Response'].mean().sort_values(ascending=False)

# Display the most successful campaigns
campaign_success.head()


,Response
CampaignID,
6,0.754808
28,0.696429
47,0.668919
15,0.652850
46,0.621495


In [7]:
# Analyze the products promoted by the most successful campaigns
# First, identify transactions associated with the most successful campaigns

# Filter the data for the top 5 most successful campaigns
successful_campaigns = [6, 28, 47, 15, 46]
successful_campaign_data = merged_df[merged_df['CampaignID'].isin(successful_campaigns)]

# Group by CampaignID and ProductID to see which products were promoted in these successful campaigns
products_promoted_by_campaign = successful_campaign_data.groupby(['CampaignID', 'ProductID']).size().unstack().fillna(0)

# Display the products promoted by the most successful campaigns
products_promoted_by_campaign.head()


ProductID,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0
CampaignID,,,,,,,,,,,,,,,,,,,,,
6,1.0,3.0,3.0,7.0,2.0,1.0,0.0,4.0,0.0,1.0,...,0.0,2.0,1.0,2.0,5.0,1.0,2.0,1.0,3.0,3.0
15,2.0,1.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0,...,6.0,1.0,1.0,3.0,1.0,3.0,1.0,1.0,1.0,2.0
28,3.0,1.0,5.0,2.0,0.0,2.0,0.0,1.0,2.0,4.0,...,3.0,3.0,2.0,2.0,2.0,0.0,3.0,2.0,2.0,2.0
46,2.0,4.0,2.0,6.0,4.0,2.0,0.0,1.0,3.0,2.0,...,0.0,5.0,4.0,1.0,2.0,3.0,3.0,3.0,2.0,0.0
47,0.0,2.0,2.0,3.0,2.0,1.0,2.0,1.0,1.0,2.0,...,1.0,1.0,3.0,1.0,1.0,2.0,1.0,3.0,3.0,0.0


In [8]:
# Analyze overall product popularity based on the number of transactions for each product
product_popularity = merged_df.groupby('ProductID')['TransactionID'].count().sort_values(ascending=False)

# Display the top 10 most popular products based on transaction count
top_10_products = product_popularity.head(10)

# Display the result
top_10_products


,TransactionID
ProductID,
78.0,138
4.0,136
31.0,132
56.0,131
5.0,131
72.0,131
81.0,131
92.0,128
22.0,125
